![Alt text](http://www.ucm.es/logo/ucm.png "a title")

<div align="center"> 
<font size=5> Máster en Nuevas Tecnologías Electrónicas y Fotónicas </font>
</div>
    
<div align="center"> 
<font size=4> Óptica Digital, curso 2022-2023 </font>
</div>

<div align="center"> 
<font size=4>  

</font>
</div>

<div align="center"> 
<font size=5> Ejercicio 5 - Elementos ópticos difractivos </font>
</div>

- **Fecha**: 
        
- **Alumno**:  Alex Recuenco 


# Introducción

En este ejercicio nos vamos a centrar en el uso de kinoformas. Éstas producen una distribución de intensidad deseada en el campo lejano. Utilizaremos dos estrategias:

- La función fft de la clase, que directamente calcula la distribución en campo lejano.
- El uso de una lente y propagar hasta la distancia focal. Esto nos permitirá el control del tamaño cambiando la focal.

Veremos el efecto de una kinoforma de amplitud y una de fase.

# Enunciados

1. Sea una kinoforma proporcionada mediante un dibujo .bmp.

    a. Cargue la imagen (DOEucm.bmp) y genere una transmitancia de amplitud. Hay una función específica dentro de Scalar_mask_XY.
    
    b. Obtenga la distribución de intensidad en campo lejano cuando esta kinoforma es iluminada por una onda armónica plana en incidencia normal.
    
    c. Convierta la imagen en una transmitancia de fase. Obtenga de nuevo la distribución de intensidad en campo lejano.
    
    d. Para obtener el campo lejano, utilice una lente. Obtenga la distribución de intensidad en el plano focal de la lente. 
    
    e. Para el caso d), utilice una máscara para eliminar el orden 0.
    
    f. Compare los resultados.
    
    
    

<div class="alert alert-block alert-success">
    
<b>Nota:</b>
    
En todos los casos, especialmente en las kinoformas de amplitud, es conveniente eliminar el orden 0 de la intensidad obtenida, pues reduce la visibilidad del resultado.

    - Para la función fft, hay un parámetro específico.

    - Para campo cercano es necesario crear la máscara y multiplicarla por el resultado.
    
</div>



Proponga un valor para los parámetros que no vengan en el enunciado. En todos los casos, los resultados son gráficas de intensidad y fase luminosa.

<div class="alert alert-block alert-info">

<b>Consejos:</b>
    
    - Para una mejor observación y rapidez en los cálculos, utilice una máscara de 512 x 512 pixeles.
    
    - El tamaño de la máscara puede ser 1 mm.
    
    - Para enmascarar, utilice una abertura circular, donde el centro tenga una transmitancia 0 y los bordes una transmitancia 1.
    
  
</div>

In [ ]:
import cv2 as cv
import numpy as np


def load_img(file):
    img = cv.imread(file, cv.IMREAD_GRAYSCALE)
    return img


img_file = "DOEucm.bmp"

## Resolución


In [ ]:
from matplotlib import pyplot as plt
from numpy.typing import NDArray

loaded_img = load_img(img_file)
plt.imshow(loaded_img, cmap="gray")

In [ ]:
(resolution, resolution_sim_y) = np.array(loaded_img.shape) * 2
assert resolution == resolution_sim_y, "Picture needs to be square"

print(resolution)
# normalize
def _normalize_img(img: NDArray):
    img = img.astype(float)
    min_val = img.min()
    max_val = img.max()

    img = (img - min_val) / (max_val - min_val)
    return img


img = _normalize_img(loaded_img)
plt.imshow(img, cmap="gray")

In [ ]:
resized_mask = cv.resize(
    img, dsize=(resolution, resolution), interpolation=cv.INTER_NEAREST_EXACT
)
plt.imshow(resized_mask, cmap="gray")

### Definiciones iniciales

In [ ]:
from os import remove
from diffractio import mm, um
from diffractio.scalar_sources_XY import Scalar_source_XY
from diffractio.scalar_masks_XY import Scalar_mask_XY

wavelength = 700 * um
x = np.linspace(-mm, mm, resolution)
y = x.copy()


def propagate(image: NDArray, **propagation_kw):
    wave = Scalar_source_XY(x, y, wavelength=wavelength)
    wave.plane_wave()
    mask = Scalar_mask_XY(x, y, wavelength=wavelength)
    mask.u = np.array(image)

    input = wave * mask
    propagation_kw.pop("new_field", None)
    far_field = input.fft(**propagation_kw, new_field=True)
    return (input, far_field)

### b. Distribución de intensidad en campo lejano

In [ ]:
input, far_field = propagate(resized_mask, remove0=False)

input.draw(normalize=True)
plt.show()
far_field.draw(logarithm=True)
plt.show()

### c. Máscara de fase

Como la mascara es binaria, podemos reemplazar 0 por 1, y 1 por -1 (Cambiando el argumento por fase). 

Este cambio se puede hacer con una transformacion lineal, que es mas rapida. Primero multiplicamos por -2 y luego sumamos 1

In [ ]:
phase_mask = resized_mask * (-2) + 1

In [ ]:
input, far_field = propagate(phase_mask, remove0=False)

input.draw(normalize=True, kind="phase")
plt.show()
far_field.draw(logarithm=True)
plt.show()

### d. Uso de una lente

In [ ]:
x = np.linspace(-6 * mm, 6 * mm, 1024)
y = x.copy()
wavelength = 0.6 * um


def propagate_lense(image: NDArray, focal=5 * mm, **propagation_kw):
    wave = Scalar_source_XY(x, y, wavelength=wavelength)
    wave.plane_wave(A=10)
    mask = Scalar_mask_XY(x, y, wavelength=wavelength)
    resized_image = cv.resize(
        image, dsize=mask.u.shape, interpolation=cv.INTER_NEAREST_EXACT
    )
    mask.u = np.array(resized_image)

    input = wave * mask
    lens_field = input
    print("\n")
    lens = Scalar_mask_XY(x, y, wavelength=wavelength)

    lens.lens((0, 0), focal, radius=0)

    lens_field = lens_field * lens

    propagation_kw.pop("new_field", None)
    focal = propagation_kw.pop("z", focal)
    far_field = lens_field.RS(**propagation_kw, z=focal, new_field=True)
    return (input, far_field)


input, far_field = propagate_lense(phase_mask, focal=500 * mm, verbose=True)

input.draw(logarithm=True)
plt.show()
far_field.draw(logarithm=True)

plt.show()

### e. Máscara para orden 0

In [ ]:
zero_mask = Scalar_mask_XY(x, y, wavelength=wavelength)
zero_mask.circle((0, 0), 750 * um)
zero_mask.u = 1 - zero_mask.u

zero_mask.draw()
plt.show()
field_zero_masked = far_field * zero_mask
field_zero_masked.draw(logarithm=True)

plt.show()

**Conclusiones**:

La FFT es un algoritmo que es O(N). Teniendo en cuenta que es una aproximación, sólo hay que tener cuidado en que régimen estamos trabajando, y a lo mejor una simulación de RS nos permite comparar nuestros resultados. Pero en general, usar la FFT ofrece mejor resolución y velocidad de computación